In [2]:
from pathlib import Path
BASE_PATH = Path("/home/anto/Downloads/3dim_res")
EXP_TIMESTAMP = "1683636212.3794007"

model_uuid = "4029cfb3-e2b7-40e9-bdcc-9a8fa3151e98"
dataset_id = "358b2eae819d51fd894ac568be097d90" # 0dca4f0d524c6f06134b63c6396e360e   358b2eae819d51fd894ac568be097d90

In [3]:
import pickle
from experiments.synthetic_datasets.common import load_dataset


with open(BASE_PATH / "results" / "models" / f"{model_uuid}.pkl", "rb") as fp:
    model = pickle.load(fp)
    

ds = load_dataset(BASE_PATH / f"synthetic_datasets/datasets/{dataset_id}.json")

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#indices = np.random.randint(len(ds.X), size=1000)
#X,y = ds.X[indices], ds.y[indices]

scaler = StandardScaler()
X = scaler.fit_transform(ds.X)

X_train, X_test, y_train, y_test = train_test_split(X, ds.y, test_size=.3)

In [5]:
from budgetsvm.kernel import Kernel

class RBFKernel(Kernel):
    def __init__(self, gamma):
        super().__init__()
        if gamma > 0:
            self.gamma = gamma
        else:
            raise ValueError()

    def compute(self, arg_1, arg_2):
        # == sklearn rbf kernel
        diff = np.linalg.norm(np.array(arg_1) - np.array(arg_2)) ** 2
        return float(np.exp(-gamma * diff))

    def __repr__(self):
        """Return the python representation of the kernel."""
        obj_repr = "RBF("
        if self.sigma != self.default_sigma:
            obj_repr += f"sigma={self.sigma}"
        obj_repr += ")"
        return obj_repr

    def __eq__(self, other):
        return isinstance(other, RBFKernel) and self.sigma == other.sigma


In [ ]:
from sklearn.svm import SVC as sklearnSVC
from budgetsvm.svm import SVC as gurobiSVC
from budgetsvm.kernel import GaussianKernel, PrecomputedKernel
from experiments.utils import Timer
import numpy as np


gamma = 1 / (len(X_train[0]) * np.var(X_train))

with Timer() as t:
    sklearn_model = sklearnSVC(C=1.0, kernel="rbf", gamma=gamma)
    sklearn_model.fit(X_train, y_train)

print(f"acc {sklearn_model.score(X_test, y_test)}\nnum_sv {sum(sklearn_model.n_support_)}")
print(f"training took {t.time}")


with Timer() as t:
    kernel = PrecomputedKernel(original_kernel=RBFKernel(gamma))
    precomputed_X_train = np.array([[kernel.compute(x, y) for y in X_train] for x in X_train])
    precomputed_X_test = np.array([[kernel.compute(x, y) for y in X_train] for x in X_test])
    
    gurobi_model = gurobiSVC(C=1.0, kernel=kernel)
    gurobi_model.fit(precomputed_X_train, y_train)

print(f"\nacc {gurobi_model.score(precomputed_X_test, y_test)}\nnum_sv {len(gurobi_model.alpha_)}")
print(f"training took {t.time}")

acc 0.9941666666666666
num_sv 596
training took 0.20316475900017394
model construction took 404.3269998780006
